In [5]:
# importing libraries
import cv2
import numpy as np
import math
# Global constant
color_distance_threshold = 8
black_threshold = 128
morphology_iterations = 3
# Line color
line_color = (100,0,255)
direction_line_color = (255,100,0)
text_location = (50,100)
err = 0.00001

In [6]:
def mid_line_equation(x_inter, y_inter, x_l, y_l, x_r, y_r):
    x_mid, y_mid = (x_l + x_r)/2, (y_l + y_r)/2
    a_mid = (y_mid - y_inter)/(x_mid-x_inter+0.00001)
    b_mid = y_mid - a_mid * x_mid
    return int(y_mid), int(x_mid), a_mid, b_mid

def detect_mid_line(ptl1, ptl2, ptr1, ptr2):
    #calculate the center line
    y_l1, x_l1, y_l2, x_l2 = ptl1[0], ptl1[1], ptl2[0], ptl2[1]
    y_r1, x_r1, y_r2, x_r2 = ptr1[0], ptr1[1], ptr2[0], ptr2[1]

    a_l, a_r = (y_l1-y_l2)/(x_l1-x_l2+0.000001), (y_r1-y_r2)/(x_r1-x_r2+0.000001)
    b_l, b_r = y_l1 - x_l1 * a_l, y_r1 - x_r1 * a_r

    #get intersect point
    x_inter = (b_r-b_l)/(a_l-a_r+0.000001)
    y_inter = x_inter*a_l + b_l

    #get mid_point left
    x_ml = x_inter + 10000
    y_ml = x_ml * a_l + b_l
    d_2 = np.square(y_inter - y_ml) + np.square(x_inter - x_ml)

    #get mid_point right
    #delta = b^2 - 4ac
    a_mr = np.square(a_r) + 1
    b_mr = -2*(x_inter+y_inter*a_r-a_r*b_r)
    c_mr = -(d_2 - np.square(x_inter) - np.square(y_inter) + 2*y_inter*b_r - np.square(b_r))
    delta = np.square(b_mr) - 4*a_mr*c_mr

    #get x_mid_right and y_mid_right
    x_mr1 = (-b_mr - np.sqrt(delta)) / (2*a_mr+0.000001)
    x_mr2 = (-b_mr + np.sqrt(delta)) / (2*a_mr+0.000001)

    y_mr1 = a_r * x_mr1 + b_r
    y_mr2 = a_r * x_mr2 + b_r

    #return mid_line
    if x_mr1 > x_inter:
        y_mid, x_mid, a_mid, b_mid = mid_line_equation(x_inter, y_inter, x_ml, y_ml, x_mr1, y_mr1)
    else:
        y_mid, x_mid, a_mid, b_mid = mid_line_equation(x_inter, y_inter, x_ml, y_ml, x_mr2, y_mr2)
    return int(y_inter), int(x_inter), y_mid, x_mid, a_mid, b_mid

In [7]:
def lane_making(img):
    # Gaussian filter to slightly reduce noise
    img_blurred = cv2.GaussianBlur(img, ksize=(3,3), sigmaX=1, sigmaY=1)
    # Convert to Lab
    img_blurred = cv2.cvtColor(img_blurred, cv2.COLOR_BGR2Lab)
    # Seperate the color components and lightning
    lightning = img_blurred[:, :, 0]
    mean_lightning = np.mean(lightning)
    color = img_blurred[:, :, 1:] - 127
    color_dist = np.sqrt(np.sum(color.__pow__(2), axis=2))
    # Apply adaptive thresholding for lightning matrix and global thresholding for color matrix
    # We must filter black pixel before feeding to otsu
    lightning = np.where(lightning>black_threshold, lightning, int(mean_lightning))
    _, lightning_thresh = cv2.threshold(lightning, 0, 255, cv2.THRESH_OTSU)
    color_thresh = color_dist < color_distance_threshold
    mixture = np.logical_and(color_thresh, lightning_thresh.astype(bool))
    thresh_result = np.where(mixture, 255, 0).astype(np.int16)
    # Apply morphology transformation to filter noises
    morph = cv2.dilate(thresh_result, kernel=(3,3), iterations=morphology_iterations)
    morph = cv2.erode(morph, kernel=(3,3), iterations=morphology_iterations)
    # Apply median filter to remove excessive noises
    bw = cv2.medianBlur(morph, ksize=5)
    # Applying the Canny Edge filter
    edges = cv2.Canny(np.uint8(bw), 0, 255)
    lines = cv2.HoughLines(edges, 1, np.pi / 90, 50, None, 0, 0)
    if lines is not None:
        len_line = len(lines)
        if len_line > 3:
            get_min = False
            get_max = False
            #draw lines
            min_pt = [len(img[0]), 0, 0]
            max_pt = [0, 0, 0]
            #get 2 main lines
            for i in range(0, len(lines)):
                rho, theta = lines[i][0][0], lines[i][0][1]
                a, b = math.cos(theta), math.sin(theta)
                x0, y0 = a * rho, b * rho
                pt1, pt2 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a))), (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))

                if abs(pt1[1]-pt2[1]) >30:
                    if min_pt[0] > rho:
                        min_pt[0], min_pt[1], min_pt[2]  = x0, pt1, pt2
                        get_min = True
                    if max_pt[0] < rho:
                        max_pt[0], max_pt[1], max_pt[2]  = x0, pt1, pt2
                        get_max = True
                else:
                    len_line -= 1

            #get center line
            if len_line > 2 and get_min and get_max:
                #get center
                center = detect_mid_line(min_pt[1], min_pt[2], max_pt[1], max_pt[2])
                x_inter, y_inter = center[1], center[0]
                a_mid, b_mid = center[4], center[5]
                if x_inter < 0.5 * len(img):
                    cv2.putText(img, ' y = {a}x + {b}'.format(a = round(a_mid,3), b = round(b_mid,3)), text_location, cv2.FONT_HERSHEY_SIMPLEX, 1, line_color, 2)
                    cv2.line(img, min_pt[1], min_pt[2], line_color, 3, cv2.LINE_AA)
                    cv2.line(img, max_pt[1], max_pt[2], line_color, 3, cv2.LINE_AA)
                    cv2.line(img, (y_inter, x_inter), (center[2], center[3]), direction_line_color, 3, cv2.LINE_AA)
                    cv2.imshow('Frame', img)
                    return

    cv2.putText(img, 'None', text_location, cv2.FONT_HERSHEY_SIMPLEX, 1, line_color, 2)
    cv2.imshow('Frame', img)
    return

In [8]:
# Create a VideoCapture object and read from input fileqqqqq
cap = cv2.VideoCapture('../../data/line_trace/congthanh_solution.mp4')
# Check if camera opened successfully
if not cap.isOpened():
    print("Error opening video file")
# Read until video is completed
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    lane_making(frame)
# When everything done, release the video capture object & Closes all the frames
cap.release()
cv2.destroyAllWindows()

Mid line equation:
y = -0.6249103430663066x + 1042.648946125104
theta = 1.0122614864510382, rho = 884.2001652167116

Mid line equation:
y = 0.034812525055975624x + 758.5767152298713
theta = -1.5359978547595254, rho = -758.1174685382201

Mid line equation:
y = 0.07017585690192936x + 929.9978094518095
theta = -1.5007353279009041, rho = -927.7162755496116

Mid line equation:
y = -0.6005775625262324x + 1085.6416594585362
theta = 1.0299522564051549, rho = 930.6928118994919

Mid line equation:
y = 2.2075442658980006e-07x + 840.1601426044922
theta = -1.57079610604047, rho = -840.1601426044717

Mid line equation:
y = 0.034812546582210877x + 760.0782997714865
theta = -1.5359978332593465, rho = -759.6181434433907

Mid line equation:
y = -0.2676001840410466x + 645.0684681056382
theta = 1.309322596659558, rho = 623.142630529365

Mid line equation:
y = -0.383595276168274x + 974.9278376110328
theta = 1.2045114477504786, rho = 910.2553692546763

Mid line equation:
y = 0.017289821630776848x + 774.1022